In [2]:
!pip install praw

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import praw


reddit = praw.Reddit(
    client_id="eu_W5FVaWlrBz8lydJMjcA",          
    client_secret="AFirlIb_Lic-bo_VHA1Zpue32e54Yw",  
    user_agent="Scraper"         
)

In [6]:
subreddit_name = "stocks"  # Choose the subreddit
query = "stock market"     # Search query (optional)


In [8]:
posts = []
subreddit = reddit.subreddit(subreddit_name)
for post in subreddit.search(query, limit=100):  # Change limit as needed
    posts.append({
        "Title": post.title,
        "Body": post.selftext,
        "Score": post.score,
        "Comments": post.num_comments,
        "Timestamp": post.created_utc
    })

In [14]:
df_reddit = pd.DataFrame(posts)

In [16]:
df_reddit.to_csv("reddit_data.csv", index=False)
print("Reddit data saved!")

Reddit data saved!


In [18]:
# Load the data
df_reddit = pd.read_csv("reddit_data.csv")

# Drop duplicates
df_reddit.drop_duplicates(subset=["Title", "Body"], inplace=True)

# Fill missing values
df_reddit.fillna("", inplace=True)

# Combine Title and Body for analysis
df_reddit["Content"] = df_reddit["Title"] + " " + df_reddit["Body"]

# Save the cleaned data
df_reddit.to_csv("cleaned_reddit_data.csv", index=False)
print("Data cleaned and saved!")


Data cleaned and saved!


In [22]:
from textblob import TextBlob

In [24]:
# Sentiment Analysis
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

df_reddit["Sentiment"] = df_reddit["Content"].apply(analyze_sentiment)

# Classify sentiment as Positive, Neutral, or Negative
df_reddit["Sentiment_Label"] = df_reddit["Sentiment"].apply(
    lambda x: "Positive" if x > 0 else ("Negative" if x < 0 else "Neutral")
)

# Save the sentiment analysis results
df_reddit.to_csv("sentiment_reddit_data.csv", index=False)
print("Sentiment analysis completed and saved!")


Sentiment analysis completed and saved!


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Prepare data for modeling
df_reddit["Stock_Movement"] = (df_reddit["Sentiment"] > 0).astype(int)  # Simplified assumption
X = df_reddit[["Sentiment", "Score", "Comments"]]
y = df_reddit["Stock_Movement"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        14

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

